<a href="https://colab.research.google.com/github/eedduu/TFG/blob/main/code/old_playgorunds/SHADE_B_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ucimlrepo
!pip install fastai
!pip install nbdev

     |████████████████████████████████| 162 kB 2.2 MB/s 
  Attempting uninstall: certifi
    Found existing installation: certifi 2019.11.28
    Uninstalling certifi-2019.11.28:
      Successfully uninstalled certifi-2019.11.28


In [ ]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from nbdev.showdoc import *
from fastai.vision.all import *
import torch.nn.functional as F
import torchvision.transforms.functional as FF
import numpy
import torchvision
from torchvision import *
import fastai
import fastcore
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from fastai.tabular.all import TabularDataLoaders, TabularPandas, RandomSplitter, accuracy

In [ ]:
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

# metadata
#print(breast_cancer_wisconsin_diagnostic.metadata)

# variable information
#print(breast_cancer_wisconsin_diagnostic.variables)


df=pd.concat([X, y], axis=1)

#X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3)


# Define the categorical and continuous variables
cat_names = []
#cont_names = [f'feature_{i}' for i in range(1, 31)]
#cont_names = []
#cont_names += ['texture1','perimeter1','area1','smoothness1','compactness1','concavity1','concave_points1','symmetry1','fractal_dimension1','texture2','perimeter2','area2','smoothness2','compactness2','concavity2','concave_points2','symmetry2','fractal_dimension2','texture3','perimeter3','area3','smoothness3','compactness3','concavity3','concave_points3','symmetry3','fractal_dimension3']
cont_names = list(X.columns)
dep_var = 'Diagnosis'

splits=RandomSplitter(valid_pct = 0.3)(range_of(df))

# Create TabularPandas
to = TabularPandas(
    df,
    procs=[Categorify, FillMissing, Normalize],
    #cat_names=cat_names,
    cont_names=cont_names,
    y_names=dep_var,
    splits=splits
)

#INFORED: He hecho varias pruebas pero hay que tener en cuenta que el dataset es pequeño con lo que un tamaño grande es contraproducente
dls = to.dataloaders(32)

In [ ]:
#layers = [1024, 1024, 512, 512, 256, 256, 128, 128, 64, 64, 32, 32, 16, 16, 8]
layers = [32]
LEARN = tabular_learner(dls, layers=layers)

In [ ]:
LEARN.summary()

TabularModel (Input shape: 32 x 0)
Layer (type)         Output Shape         Param #    Trainable 
                     32 x 30             
BatchNorm1d                               60         True      
____________________________________________________________________________
                     32 x 1024           
Linear                                    30720      True      
ReLU                                                           
BatchNorm1d                               2048       True      
Linear                                    1048576    True      
ReLU                                                           
BatchNorm1d                               2048       True      
____________________________________________________________________________
                     32 x 512            
Linear                                    524288     True      
ReLU                                                           
BatchNorm1d                               102

In [ ]:
import sys
sys.path.append('/content/pyade-master')

!pip install '/content/drive/MyDrive/pyade-master'

import pyade

Processing ./drive/MyDrive/pyade-master
  Preparing metadata (setup.py) ... done
  Created wheel for pyade-python: filename=pyade_python-1.0-py3-none-any.whl size=29040 sha256=f09c8eb2b3eb7943bc02703bc80e702953ae1db2136685b74fa0adba336980e1
  Stored in directory: /root/.cache/pip/wheels/89/b8/51/ff2ebef24405d3ef46ccb451f962b374b2745f263fb000c434
Successfully built pyade-python
  Attempting uninstall: pyade-python
    Found existing installation: pyade-python 1.0
    Uninstalling pyade-python-1.0:
      Successfully uninstalled pyade-python-1.0


In [ ]:
#De un diccionario ordenado de fastai a array de parametros
def get_params_from_state_dict(state_dict):
    return np.concatenate([v.cpu().numpy().ravel() for v in state_dict.values()])

def get_params_from_model(model):
    return np.concatenate([v.cpu().detach().numpy().ravel() for v in model.parameters()])


#De array de parametros a diccionario ordenado
def set_params_to_state_dict(params):
    state_dict = LEARN.model.state_dict()
    offset = 0
    for key, param in state_dict.items():
        param_size = param.numel()
        state_dict[key] = torch.tensor(params[offset:offset + param_size]).view(param.size())
        offset += param_size
    return state_dict


#Funcion de error que recibe un array de parametros
def err_arr(params):
    weights = set_params_to_state_dict(params)
    LEARN.model.load_state_dict(weights)
    LEARN.train()
    preds, targ, losses = LEARN.get_preds(dl=dls.train, with_loss=True)

    return losses.mean().item()


def set_params_to_model(params):
    offset = 0
    for param in LEARN.model.parameters():
        param_size = param.numel()
        param.data = torch.tensor(params[offset:offset + param_size], dtype=param.data.dtype).view(param.size())
        param.requires_grad = True
        offset += param_size

def err_param(params):
    set_params_to_model(params)

    LEARN.train()
    #Option 1: calculate the error per each batch and make the mean

    loss =0.0
    for batch in dls.train:
        # Handle the case where there are no continuous features
        x_cont = batch[1] if len(batch) > 1 and batch[1] is not None else None
        pred = LEARN.model(batch[0], x_cont)  # Pass both categorical and continuous features (if any)
        target = batch[2].squeeze().long()
        loss += F.cross_entropy(pred, target)

    loss/=len(dls.train)


    return loss.mean().item()








In [ ]:
par = np.concatenate([param.cpu().detach().numpy().ravel() for param in LEARN.model.parameters()])
params_flat = get_params_from_state_dict(LEARN.model.state_dict())

In [ ]:
import pyade.shade

algorithm = pyade.shade

def SHADE_ej(population, pop_size, max_evals, dim, prevm_cr=0, prevm_f=0):

  population = np.array(population).reshape(pop_size, dim)

  params = algorithm.get_default_params(dim=dim)
  # We define the boundaries of the variables
  params['bounds'] = np.array([[-50, 50]] * dim)

  # We indicate the function we want to minimize
  params['func'] = err_param

  params['population_size']=pop_size



  params['seed']=42




  params['init_pop']=population

  params['max_evals']=max_evals


  params['prevm_cr']=prevm_cr

  params['prevm_f']=prevm_f

  return algorithm.apply(**params)






- Tengo SHADE funcionando. Tengo que hace ILS-SHADE basicamente añadiendo la busqueda local y todo el rollo según el paper de dani molina.

- He cambiado el return del shade.py para que devuelva toda la poblacion, todo el vector de fitness y que ademas devuelva los parametros, y que se puedan volver a meter luego para que se siga adaptando. Solo guardo los parametros de las matrices porque son los que guardan los resultados exitosos de los valores, y los que se aprovechan luego en el resto de ejecuciones. Podría rentar meter la ejecucion de SHADE junto con el correspondiente guardado


- Tengo funcionando la Búsqueda local con L-BFGS-B. Tengo dos opciones: la implementación con pytorch y la implementación con scipy. La de scipy calcula el gradiente aproximado de un batch y hace bastantes iteraciones (unas 30) mientras que la otra la calcula en minibatches y aplica cambios tras cada mini-batch y solo hace un epoch.

- La de PyTorch en principio está más pensada para este tipo de optimizaciones de machine learning. Al principio funcionaba mucho mejor la de PyTorch pero ahora funciona mucho mejor la de ScipY

- Sería interesante y justificable usar solo esta búsqueda local en el algoritmo de SHADE-ILS ya que es un método conocido de segundo orden para entrenamiento de modelos.

- Cuando vaya a entrenar a resnet18 será complicado por el número de parámetros. Opciones varias son: meter menos búsqueda local en ese momento, buscar un modelo más pequeño, buscar una búsqueda local más accesible, o, en última opción: no hacer SHADE-ILS con resnet18. He probado con el MLP de 15 capas y no tarda demasiado obteniendo resultados aceptables de 0.17 (3 min)

- Ejecucion de SHADE-ILS con MLP 15 capas: 14 min

- Aun asi puedo investigar un poco el otro tipo de BL: MTS LS

# TODO

1. AÑADIR la opción del restart a SHADE-ILS: criterio de restart y acciones

2. (Opcional) AÑADIR metodo MTS LS. En tal caso tengo que añadir también el criterio para que se aplique o no la LS y cual se aplica.

3. FINE-TUNE de parámetros: evaluaciones, num_pob, etc

4. REDACTAR la experimentacion detalladamente y enviarsela a pablo.

5. PENSAR qué métricas, datos voy a medir y a comparar.

6. COMPROBAR cálculo del gradiente y función de error.

7. (OPCIONAL) COMPROBAR qué pasa usando la función f\_min\_fbgs de minimize de scipy: es en esencia la misma función que la que estoy usando ahora pero no usa ninguna función gradiente: lo hace a través de una aproximación por diferencias finitas. Intuición de lo que va a pasar: si uso minimize() sin pasarle la función gradiente para la aproximación tarda una eternidad (para de 1seg a más de 4min para 32 neuronas)

In [ ]:
#BUSQUEDA LOCAL CON LBFGS CON LIBRERIA SCIPY
from scipy.optimize import minimize
LEARN = tabular_learner(dls, layers=layers)
def get_gradient(ind):
    set_params_to_model(ind)
    LEARN.train()  # Put the model in training mode
    batch = dls.train.one_batch()  # Get one batch of data

    # Handle the case where there are no continuous features
    x_cont = batch[1] if len(batch) > 1 and batch[1] is not None else None

    pred = LEARN.model(batch[0], x_cont)  # Pass both categorical and continuous features (if any)

    target = batch[2].squeeze().long()


    loss = F.cross_entropy(pred, target)  # Calculate loss (assuming target labels are at index 2)
    loss.backward()  # Calculate gradients

    # Extract gradients into a list
    gradients = [param.grad.clone().detach().cpu().numpy().ravel() for param in LEARN.model.parameters()]

    # Concatenate gradients into a single 1D array
    return np.concatenate(gradients)


def LBFGS_SCIPY(ind):
    result = minimize(err_param, ind, jac=get_gradient, method='L-BFGS-B')
    return result.x, result.fun, result.nfev, result.njev


In [ ]:
import torch.optim as optim
layers=[64, 32, 8]
LEARN =  tabular_learner(dls, layers=layers)



def epoch_LBFGS(ind):
    optimizer = optim.LBFGS(LEARN.model.parameters(), lr=0.01)
    set_params_to_model(ind)
    LEARN.train()  # Put the model in training mode


    def closure():
        optimizer.zero_grad()  # Clear previous gradients
        batch = next(iter(dls.train))  # Get one batch of data
        # Handle the case where there are no continuous features
        x_cont = batch[1] if len(batch) > 1 and batch[1] is not None else None
        output = LEARN.model(batch[0], x_cont)  # Forward pass
        target = batch[2].squeeze().long()
        loss = F.cross_entropy(output, target)  # Accumulate loss
        loss.backward()  # Backward pass after accumulating loss over all batches
        return loss

    for _ in range(len(dls.train)): # Assuming you want to run for one epoch
        optimizer.step(closure)
        print("EO")

    return get_params_from_model(LEARN)

ind=get_params_from_model(LEARN)
print(err_param(ind))
ind = epoch_LBFGS(ind)
print(err_param(ind))

In [ ]:
#Algoritmo SHADE-ILS
from collections import deque
#layers=[1024, 1024,512, 512, 256, 256, 128, 128, 64, 64, 32, 32, 8, 8, 4]
layers=[32]
LEARN =  tabular_learner(dls, layers=layers)

#layers=[32]


population_size = 25
evals = 0
max_evals = 3000
max_evals_SHADE = 100
prevm_cr=0
prevm_f=0
must_restart=False
improq = deque([1,1,1], maxlen=3)

#Initialize the population
population = [get_params_from_model(tabular_learner(dls, layers=layers)) for _ in range(population_size)]
fitness = [err_param(elem) for elem in population]
evals += population_size
size_ind=len(population[0])

#Select the best

current_best_fitness = min(fitness)
current_best_index = fitness.index(current_best_fitness)
current_best = population[current_best_index]

temp_current_best, temp_current_best_fitness, e1, e2 = LBFGS_SCIPY(current_best)
evals += e1 + e2

improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

if temp_current_best_fitness < current_best_fitness:
  current_best_fitness = temp_current_best_fitness
  current_best = temp_current_best


population[current_best_index] = current_best
fitness[current_best_index] = current_best_fitness

best_solution=current_best
best_fitness=current_best_fitness

while evals < max_evals:
  print("eo")
  population, fitness, prevm_cr, prevm_f = SHADE_ej(population, population_size, max_evals_SHADE, size_ind, prevm_cr=prevm_cr, prevm_f=prevm_f)
  evals += max_evals_SHADE
  temp_current_best_fitness= min(fitness)

  improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

  if temp_current_best_fitness < current_best_fitness:
    current_best_fitness = temp_current_best_fitness
    current_best_index = list(fitness).index(current_best_fitness)
    current_best = population[current_best_index]



  print(improq)
  if np.all(np.array(improq)<0.05):
    must_restart=True

  #Choose the LS method to apply this iteration based on improvement

  temp_current_best, temp_current_best_fitness, e1, e2 = LBFGS_SCIPY(current_best)
  evals += e1 + e2

  improq.append((current_best_fitness-temp_current_best_fitness)/current_best_fitness)

  if temp_current_best_fitness < current_best_fitness:
    current_best_fitness = temp_current_best_fitness
    current_best = temp_current_best


  print(improq)
  if np.all(np.array(improq)<0.05):
    must_restart=True

  population[current_best_index] = current_best
  fitness[current_best_index] = current_best_fitness

  #Update the probability to apply LS in next iterations

  if current_best_fitness < best_fitness:
    best_fitness = np.copy(current_best_fitness)
    best_solution = np.copy(current_best)

  if must_restart:
    random_index = np.random.choice(population_size)
    sol = population[random_index]
    sol += np.random.normal(0, 100, len(sol))

    population = [get_params_from_model(tabular_learner(dls, layers=layers)) for _ in range(population_size-1)]
    fitness = [err_param(elem) for elem in population]
    evals += population_size
    population.append(sol)
    current_best_fitness=err_param(sol)
    current_best=sol
    fitness.append(current_best_fitness)
    improq=deque([1,1,1], maxlen=3)

    print("Reiniciado")
    must_restart=False


eo
deque([1, 0.5873470962560707, 0.40669233524140797], maxlen=3)
deque([0.5873470962560707, 0.40669233524140797, 0.07956459098364536], maxlen=3)
eo
deque([0.40669233524140797, 0.07956459098364536, 0.09693808393611848], maxlen=3)
deque([0.07956459098364536, 0.09693808393611848, -0.106735860275979], maxlen=3)
eo
deque([0.09693808393611848, -0.106735860275979, 0.0], maxlen=3)
deque([-0.106735860275979, 0.0, -0.35898917922362844], maxlen=3)
Reiniciado
eo
deque([1, 1, 0.9999785176510713], maxlen=3)
deque([1, 0.9999785176510713, 0.56348202421379], maxlen=3)
eo
deque([0.9999785176510713, 0.56348202421379, 0.13888879258625697], maxlen=3)
deque([0.56348202421379, 0.13888879258625697, -0.008993162160511393], maxlen=3)
eo
deque([0.13888879258625697, -0.008993162160511393, 0.05285602593810888], maxlen=3)
deque([-0.008993162160511393, 0.05285602593810888, -0.23086261165990693], maxlen=3)
eo
deque([0.05285602593810888, -0.23086261165990693, -0.03924239219686979], maxlen=3)
deque([-0.2308626116599069

In [ ]:
print(fitness)

[4.90693063e-01 4.29877877e-01 4.83357102e-01 4.05508637e-01
 5.36503375e-01 3.84969562e-01 4.46414798e-01 2.73586065e-01
 6.98655605e-01 5.24204969e-01 4.42965508e-01 4.32018399e-01
 7.27365792e-01 6.29636586e-01 5.93505919e-01 6.76367044e-01
 4.07230705e-01 4.44136024e-01 7.54291952e-01 5.67448914e-01
 5.35882771e-01 4.22633529e-01 6.19049489e-01 6.22650683e-01
 6.73533740e+03]
